In [6]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import pandas as pd
import numpy as np

import load_data

In [7]:
tcga = load_data.read_data_sets("./data/mRNA_PCA_20_components.csv", "label_tissue")

### Simple softmax

In [8]:
# Input 20 features (60% PCA variance), output 32 tissue types
x = tf.placeholder(tf.float32, [None, 20])
W = tf.Variable(tf.zeros([20, 32]))
b = tf.Variable(tf.zeros([32]))
y = tf.nn.softmax(tf.matmul(x, W) + b)

In [9]:
y_ = tf.placeholder(tf.float32, [None, 32])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [12]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
init.run()
for i in range(1000):
    batch_xs, batch_ys = tcga.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    if i % 100 == 0:
        correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        train_accuracy = sess.run(accuracy, feed_dict={x: batch_xs, y_: batch_ys})
        test_accuracy = sess.run(accuracy, feed_dict={x: tcga.test.X, y_: tcga.test.y})
        print("Iteration={0}, train accuracy={1}, test acurracy={2}".format(
            i, train_accuracy, test_accuracy))

Iteration=0, train accuracy=0.7099999785423279, test acurracy=0.5477293729782104
Iteration=100, train accuracy=0.009999999776482582, test acurracy=0.011121409013867378
Iteration=200, train accuracy=0.0, test acurracy=0.011121409013867378
Iteration=300, train accuracy=0.019999999552965164, test acurracy=0.011121409013867378
Iteration=400, train accuracy=0.0, test acurracy=0.011121409013867378
Iteration=500, train accuracy=0.0, test acurracy=0.011121409013867378
Iteration=600, train accuracy=0.009999999776482582, test acurracy=0.011121409013867378
Iteration=700, train accuracy=0.029999999329447746, test acurracy=0.011121409013867378
Iteration=800, train accuracy=0.0, test acurracy=0.011121409013867378
Iteration=900, train accuracy=0.009999999776482582, test acurracy=0.011121409013867378


### one hidden layer

In [9]:
def weight_variable(shape):
    initial=tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [13]:
len(set(y))

32

In [23]:
X.shape

(10799, 20)